Failing Jobs in the Past 7 Days for all SQL Server Instances

In [2]:
# Read data from SQL Server
import pyodbc
import sqlalchemy as sa
import numpy as np
import pandas as pd
from sqlalchemy.engine import URL       # To support pandas
from sqlalchemy import create_engine    # To support pandas
import openpyxl
from openpyxl.styles import PatternFill
import xlsxwriter
#import seaborn as sns
#import matplotlib.pyplot as plt
#from IPython.display import display

# Enter no. of days for report
days=7
#days = input("Enter no. of days to search: ")
#print(days)

appended_data = []
file='c:\\aaa\\SQL_Servers_Failed_Jobs.xlsx'

for server in ('USSQL', 'BSQL', 'KSSQL', 'ZSSQL', 'WSSQL', 'SQLCLUST','BIDEV16','DWH16-AZURE'):
    # server parameters
    # server = 'ZSSQL' 
    database = 'DBA' 
    username = 'naya5' 
    password = '9451P9Qt%&h!'
    conn_string =  'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
    
    conn = pyodbc.connect(conn_string)
    conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string}) # To support pandas

    query = (f"SELECT "  
                    "h.server, "
                    "j.[name] as job_name, "
                    "c.name as Category, "
                    "s.step_name, "
                    "case h.run_status "
                        " when 0 then 'Failed' "
                        " when 2 then 'Retry' "
                        " when 3 then 'Canceled' "
                        " when 4 then 'In Progress' "
                    "end as last_run_status, "
                    "MSDB.dbo.agent_datetime(h.run_date,h.run_time) as start_datetime, "
                    "h.message "
            "FROM    msdb.dbo.sysjobhistory h  "
                    "INNER JOIN msdb.dbo.sysjobs j ON h.job_id = j.job_id  "
                    "INNER JOIN msdb.dbo.sysjobsteps s ON j.job_id = s.job_id AND h.step_id = s.step_id  "
                    "LEFT JOIN msdb.dbo.syscategories c ON c.category_id = j.category_id "
            "WHERE    h.run_status != 1 "
                    "AND j.enabled = 1 "
                    "AND MSDB.dbo.agent_datetime(h.run_date,h.run_time) >= GETDATE()-" + str(days) +
            " ORDER BY h.instance_id DESC"
            )

        
    # print(query)

    # --- Print using pandas dataframe ---
    print(f'Connecting to {server}')
    engine = sa.create_engine(conn_url)
    df = pd.read_sql(query, engine)
    
    # store DataFrame in list
    appended_data.append(df) 

# appended_data
appended_data = pd.concat(appended_data)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(file, engine='xlsxwriter')

# Export DataFrame content
sheet_name='Sheet1'
appended_data.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets[sheet_name]

# Set backgrund colors depending on cell values - not working!!!
#for cell, in worksheet[f'B2:B{len(appended_data) + 1}']:                   # Skip header row, process as many rows as there are DataFrames
#        value = appended_data["a"].iloc[cell.row - 2]                      # value is "Failed"
#        cell.fill = PatternFill("solid", start_color=("5cb800" if value == "Failed" else 'ff2800'))

# Add a header format.
header_format = workbook.add_format({
        "valign": "vcenter",
        "align": "left",
        "bg_color": "#951F06",
        "bold": True,
        "font_color": "#FFFFFF",
        "border" : 1, 
        "border_color": "#D3D3D3"
         })

row_format = workbook.add_format({
        'bg_color':'yellow'
        })

# Write the column headers with the defined format.
for col_num, value in enumerate(appended_data.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
    
# Add background color
# see example here:
# https://stackoverflow.com/questions/28966420/how-to-set-formatting-for-entire-row-or-column-in-xlsxwriter-python
# worksheet.set_row(3, None,  row_format)
# worksheet.set_row(4, None,  row_format)


# Manually adjust the width of column by it's position
writer.sheets['Sheet1'].set_column(1, 1, 20)
writer.sheets['Sheet1'].set_column(2, 2, 50)
writer.sheets['Sheet1'].set_column(3, 3, 25)
writer.sheets['Sheet1'].set_column(4, 4, 50)
writer.sheets['Sheet1'].set_column(5, 5, 20)
writer.sheets['Sheet1'].set_column(6, 6, 20)
writer.sheets['Sheet1'].set_column(7, 7, 200)

# Close the Pandas Excel writer and output the Excel file.
writer.close()

# Open Excel to display
import os
os.startfile(file)


Connecting to USSQL
Connecting to BSQL
Connecting to KSSQL
Connecting to ZSSQL
Connecting to WSSQL
Connecting to SQLCLUST
Connecting to BIDEV16
Connecting to DWH16-AZURE
